In [1]:
!pip install neuralforecast

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.6/170.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.9/776.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 13.1 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from neuralforecast import NeuralForecast
from neuralforecast.models import LSTM
from neuralforecast.losses.pytorch import MSE, MAE, MQLoss, DistributionLoss
from neuralforecast.tsdataset import TimeSeriesDataset, TimeSeriesLoader
from numpy.random import seed
from random import randrange
from neuralforecast.losses.numpy import rmse, mape
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [4]:
df = pd.read_excel("SP100_vol.xlsx", index_col="Date",parse_dates=True)
columns=df.columns
df_static=pd.read_excel("Static_df.xlsx")
n_series=len(df.columns)
train = df.iloc[:int(len(df['NKE'])*0.7)]
validation_length=int(len(df['NKE'])*0.7)-int(len(df['NKE'])*0.5)
train=train.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

# Hyperparameter Search

In [5]:
n_inputs = [3,5,10,15,21,42]
encoder_layers=[1,2,4,6]
dencoder_layers=[1,2,4,6]
encoder_hidden_size=[50,100,200,300,400,500]
dencoder_hidden_size=[50,100,200,300,400,500]
encoder_dropout=[0,0.2,0.3,0.4]
epochs=[50,100,150,250,350,450,550,650,750]
learning_rate=[0.0005,0.0001,0.00005,0.00001]
num_lr_decays=[5,3,2,1]
scaler_type=["robust","standard",'minmax']
context_size=[5,10,15,20,30,40]
losses=[MSE(),MAE(),MQLoss(level=[80, 90]),DistributionLoss(distribution='StudentT', level=[80, 90])]


In [5]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 127090240


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 250 | Inputs: 42 | Encoder Layers: 2 | Loss: 2 | Scaler Type: robust
Learning Rate: 0 | Encoder Dropout: 0 | Dencoder Layers: 1
Encoder Hidden Size: 300 | Decoder Hidden Size: 50 | Context Size 30
RMSE: 0.00491807938778896 | QLIKE: 0.0793694114706802 | MAE: 0.003623722181811774 | MAPE: 0.41221405799417526


In [6]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 42580966


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 750 | Inputs: 21 | Encoder Layers: 6 | Loss: 1 | Scaler Type: minmax
Learning Rate: 0 | Encoder Dropout: 0.4 | Dencoder Layers: 6
Encoder Hidden Size: 400 | Decoder Hidden Size: 500 | Context Size 30
RMSE: 0.005472353129784601 | QLIKE: 0.09289943847269554 | MAE: 0.004121697841090206 | MAPE: 0.47978804130813607


In [7]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 27810397


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 250 | Inputs: 3 | Encoder Layers: 4 | Loss: 2 | Scaler Type: standard
Learning Rate: 2 | Encoder Dropout: 0 | Dencoder Layers: 6
Encoder Hidden Size: 400 | Decoder Hidden Size: 500 | Context Size 10
RMSE: 0.004623001657402837 | QLIKE: 0.09651618136058937 | MAE: 0.003017195196242025 | MAPE: 0.2749898057475325


In [8]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 122986805


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 750 | Inputs: 15 | Encoder Layers: 4 | Loss: 1 | Scaler Type: robust
Learning Rate: 1 | Encoder Dropout: 0 | Dencoder Layers: 6
Encoder Hidden Size: 200 | Decoder Hidden Size: 100 | Context Size 15
RMSE: 0.00310481275973814 | QLIKE: 0.03512637542564985 | MAE: 0.0020099280266493444 | MAPE: 0.20414072893803278


In [9]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 103648375


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 10 | Encoder Layers: 2 | Loss: 1 | Scaler Type: standard
Learning Rate: 1 | Encoder Dropout: 0.4 | Dencoder Layers: 6
Encoder Hidden Size: 200 | Decoder Hidden Size: 500 | Context Size 15
RMSE: 0.005578202679054434 | QLIKE: 0.10158476609869768 | MAE: 0.004545896936054993 | MAPE: 0.5468340179512107


In [10]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 64162053


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 350 | Inputs: 3 | Encoder Layers: 4 | Loss: 1 | Scaler Type: standard
Learning Rate: 1 | Encoder Dropout: 0 | Dencoder Layers: 2
Encoder Hidden Size: 500 | Decoder Hidden Size: 500 | Context Size 40
RMSE: 0.04216493115636849 | QLIKE: 0.8423063908812266 | MAE: 0.03924078255338521 | MAPE: 4.537744907130497


In [11]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 106185793


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 250 | Inputs: 15 | Encoder Layers: 4 | Loss: 3 | Scaler Type: robust
Learning Rate: 0 | Encoder Dropout: 0.2 | Dencoder Layers: 1
Encoder Hidden Size: 50 | Decoder Hidden Size: 400 | Context Size 40
RMSE: 0.003931727276107689 | QLIKE: 0.05631681306219822 | MAE: 0.002655174663917594 | MAPE: 0.2654418004505696


In [12]:
for z in range (5):
  i=randrange(6)
  h=randrange(4)
  j=randrange(6)
  j2=randrange(6)
  j3=randrange(6)
  a=randrange(4)
  b=randrange(4)
  k=randrange(9)
  l=randrange(4)
  m=randrange(4)
  n=randrange(4)
  o=randrange(3)
  model=LSTM (h=1, input_size=n_inputs[i],
       encoder_n_layers=encoder_layers[h], encoder_hidden_size=encoder_hidden_size[j],
       encoder_dropout=encoder_dropout[b],
       context_size=context_size[j3], decoder_hidden_size=dencoder_hidden_size[j2],
       decoder_layers=dencoder_layers[a],
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[m],
       max_steps=epochs[k], learning_rate=learning_rate[l],
       num_lr_decays=num_lr_decays[l],
       val_check_steps=500, scaler_type=scaler_type[o],
       random_seed=randrange(129228148))
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=1,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Epoch: {epochs[k]} | Inputs: {n_inputs[i]} | Encoder Layers: {encoder_layers[h]} | Loss: {m} | Scaler Type: {scaler_type[o]}")
  print(f"Learning Rate: {l} | Encoder Dropout: {encoder_dropout[b]} | Dencoder Layers: {dencoder_layers[a]}")
  print(f"Encoder Hidden Size: {encoder_hidden_size[j]} | Decoder Hidden Size: {dencoder_hidden_size[j2]} | Context Size {context_size[j3]}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 38244568


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Epoch: 100 | Inputs: 5 | Encoder Layers: 4 | Loss: 2 | Scaler Type: robust
Learning Rate: 3 | Encoder Dropout: 0.2 | Dencoder Layers: 2
Encoder Hidden Size: 100 | Decoder Hidden Size: 400 | Context Size 20
RMSE: 0.004881380101926158 | QLIKE: 0.08351675743521447 | MAE: 0.0038189986170455753 | MAPE: 0.44646985619806295


INFO:lightning_fabric.utilities.seed:Seed set to 78504931

Epoch: 350 | Inputs: 15 | Encoder Layers: 4 | Loss: 3 | Scaler Type: robust
Learning Rate: 0 | Encoder Dropout: 0.2 | Dencoder Layers: 4
Encoder Hidden Size: 200 | Decoder Hidden Size: 300 | Context Size 30

RMSE: 0.0030010262295565603 | QLIKE: 0.03153326466177955 | MAE: 0.0019081308939873313 | MAPE: 0.19144568749707142

In [6]:
for z in range (5):
  random_seed=randrange(129228148)
  model=LSTM (h=1, input_size=15,
       encoder_n_layers=4, encoder_hidden_size=200,
       encoder_dropout=0.2,
       context_size=30, decoder_hidden_size=300,
       decoder_layers=4,
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[3],
       max_steps=350, learning_rate=learning_rate[0],
       num_lr_decays=num_lr_decays[0],
       val_check_steps=500, scaler_type='robust',
       random_seed=random_seed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=5,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Seed: {random_seed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 47232691


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Seed: 4171155
RMSE: 0.0030213703133298232 | QLIKE: 0.03344272727125931 | MAE: 0.0020129757249645916 | MAPE: 0.2129865136106683


In [7]:
for z in range (5):
  random_seed=randrange(129228148)
  model=LSTM (h=1, input_size=15,
       encoder_n_layers=4, encoder_hidden_size=200,
       encoder_dropout=0.2,
       context_size=30, decoder_hidden_size=300,
       decoder_layers=4,
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[3],
       max_steps=350, learning_rate=learning_rate[0],
       num_lr_decays=num_lr_decays[0],
       val_check_steps=500, scaler_type='robust',
       random_seed=random_seed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=5,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Seed: {random_seed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 55868197


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Seed: 49812810
RMSE: 0.002907666185796212 | QLIKE: 0.030127871572728355 | MAE: 0.0018433329006215826 | MAPE: 0.1839237747228033


In [8]:
for z in range (5):
  random_seed=randrange(129228148)
  model=LSTM (h=1, input_size=15,
       encoder_n_layers=4, encoder_hidden_size=200,
       encoder_dropout=0.2,
       context_size=30, decoder_hidden_size=300,
       decoder_layers=4,
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[3],
       max_steps=350, learning_rate=learning_rate[0],
       num_lr_decays=num_lr_decays[0],
       val_check_steps=500, scaler_type='robust',
       random_seed=random_seed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=5,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Seed: {random_seed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 2948078


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Seed: 17678468
RMSE: 0.002718095248264413 | QLIKE: 0.027244813133627782 | MAE: 0.0017362946891489894 | MAPE: 0.17878834803101837


In [9]:
for z in range (5):
  random_seed=randrange(129228148)
  model=LSTM (h=1, input_size=15,
       encoder_n_layers=4, encoder_hidden_size=200,
       encoder_dropout=0.2,
       context_size=30, decoder_hidden_size=300,
       decoder_layers=4,
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[3],
       max_steps=350, learning_rate=learning_rate[0],
       num_lr_decays=num_lr_decays[0],
       val_check_steps=500, scaler_type='robust',
       random_seed=random_seed)
  fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
  forecasts = fcst.cross_validation(df=train,val_size=5,static_df=df_static,n_windows=None, test_size=validation_length-validation_length%1,step_size=1)
  forecasts = forecasts.dropna()
  if "LSTM-median" not in list(forecasts.columns.values):
    Y_hat=forecasts["LSTM"].values
  else:
    Y_hat=forecasts["LSTM-median"].values
  Y_true=forecasts["y"].values
  RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
  QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
  MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
  MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
  print(f"Seed: {random_seed}")
  print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 97568710


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

...

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

Seed: 79753265
RMSE: 0.003115093313822682 | QLIKE: 0.03531256029925205 | MAE: 0.002092591428863863 | MAPE: 0.22185628274197094


# Test

In [10]:
test_length=int(len(df['NKE'])*0.3)
df1=df.unstack().reset_index(name="Value").rename(columns={"level_0":"unique_id", "Date":"ds", "Value":"y"})

In [11]:
h=1

In [12]:
model=LSTM (h=1, input_size=15,
       encoder_n_layers=4, encoder_hidden_size=200,
       encoder_dropout=0.2,
       context_size=30, decoder_hidden_size=300,
       decoder_layers=4,
       stat_exog_list=['AMT', 'COP', 'CVX', 'AXP', 'BAC', 'C', 'CB', 'CME', 'GS', 'JPM', 'MS',
       'PNC', 'USB', 'WFC', 'BA', 'CAT', 'GE', 'HON', 'LMT', 'MMM', 'UNP',
       'UPS', 'AAPL', 'ACN', 'ADBE', 'ADP', 'CMCSA', 'CRM', 'CSCO', 'DIS',
       'FIS', 'FISV', 'GOOG', 'IBM', 'INTC', 'INTU', 'MA', 'MSFT', 'NFLX',
       'NVDA', 'ORCL', 'QCOM', 'T', 'TXN', 'VZ', 'COST', 'KO', 'MO', 'PEP',
       'PG', 'WMT', 'D', 'DUK', 'SO', 'ABT', 'AMGN', 'BDX', 'BMY', 'BSX', 'CI',
       'CVS', 'DHR', 'GILD', 'ISRG', 'JNJ', 'LLY', 'MDT', 'MRK', 'PFE', 'SYK',
       'TMO', 'UNH', 'AMZN', 'HD', 'LOW', 'MCD', 'NKE', 'SBUX', 'TGT', 'TJX'],
       loss=losses[3],
       max_steps=350, learning_rate=learning_rate[0],
       num_lr_decays=num_lr_decays[0],
       val_check_steps=500, scaler_type='robust',
     random_seed=17678468)
fcst = NeuralForecast(models=[model],freq=CustomBusinessDay(calendar=USFederalHolidayCalendar()))
forecasts = fcst.cross_validation(df=df1,val_size=h,static_df=df_static,n_windows=None, test_size=test_length-test_length%h,step_size=h)
forecasts = forecasts.dropna()
if "LSTM-median" not in list(forecasts.columns.values):
  Y_hat=forecasts["LSTM"].values
else:
  Y_hat=forecasts["LSTM-median"].values
Y_true=forecasts["y"].values
RMSE=np.sqrt(np.sum(((Y_true-Y_hat)**2))/len(Y_true))
QLIKE=(np.sum(Y_true/Y_hat-np.log(abs(Y_true)/abs(Y_hat))-1)/len(Y_true))
MAE=np.sum(abs(Y_true-Y_hat))/len(Y_true)
MAPE=np.sum(abs(Y_true-Y_hat)/(Y_true))/len(Y_true)
print(f"RMSE: {RMSE} | QLIKE: {QLIKE} | MAE: {MAE} | MAPE: {MAPE}")

INFO:lightning_fabric.utilities.seed:Seed set to 17678468


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Predicting: |          | 0/? [00:00<?, ?it/s]

RMSE: 0.004195003958202034 | QLIKE: 0.03486473475443946 | MAE: 0.0025921352539423665 | MAPE: 0.20078472435630526


In [13]:
df1=forecasts.drop(columns=["cutoff", "y"]).set_index("ds")

In [14]:
df1.pivot(columns="unique_id").to_excel("LSTM forecasts.xlsx")